# Poker Simulator - Interactive Demo

This notebook demonstrates the poker simulator library with examples of hand evaluation, game simulation, and statistical analysis.


In [ ]:
# Import required modules
import random
from itertools import combinations
import numpy as np
from scipy import stats

from deck import Deck, Player, TexasHoldem, Card
from evaluator import best

In [ ]:
## Poker Hand Rankings

The poker hand rankings from highest to lowest are:

- **10) Royal Flush**: Ace-King-Queen-Jack-10, all of the same suit
- **9) Straight Flush**: Five consecutive cards, all of the same suit
- **8) Four of a Kind**: Four cards of the same rank
- **7) Full House**: Three of a kind plus a pair
- **6) Flush**: Five cards of the same suit
- **5) Straight**: Five consecutive cards
- **4) Three of a Kind**: Three cards of the same rank
- **3) Two Pair**: Two different pairs
- **2) One Pair**: Two cards of the same rank
- **1) High Card**: No combination - highest card wins

The evaluator algorithm:
1. Takes a player's 7 cards (2 hole + 5 community)
2. Finds all possible 5-card combinations (C(7,5) = 21 combinations)
3. Evaluates each combination for hand strength
4. Returns the best combination with rank and tiebreaker info

In [ ]:
## Example 1: Basic Game Simulation

def simulate_hands(player_hands, opp_hands, num_sims=10000):
    """
    Simulate multiple hands to estimate win rate.
    
    Args:
        player_hands: List of Card objects for player's hole cards
        opp_hands: List of Card objects for opponent's hole cards
        num_sims: Number of simulations to run
        
    Returns:
        Dictionary with win/tie/loss rates
    """
    wins, ties = 0, 0
    for _ in range(num_sims):
        deck = Deck()
        deck.cards = [card for card in deck.cards if card not in player_hands + opp_hands]
        deck.shuffle()
        community = deck.deal(5)
        
        player_rank, player_hand = best(player_hands + community)
        opp_rank, opp_hand = best(opp_hands + community)
        
        if player_rank > opp_rank:
            wins += 1
        elif player_rank == opp_rank:
            ties += 1
            
    return {
        'Win Rate': wins / num_sims,
        'Tie Rate': ties / num_sims,
        'Loss Rate': 1 - (wins + ties) / num_sims
    }

In [ ]:
# Test: Pair of Aces vs Pair of Kings
player_hand = [Card('Ace', 'Hearts'), Card('Ace', 'Spades')]
opp_hand = [Card('King', 'Hearts'), Card('King', 'Diamonds')]
results = simulate_hands(player_hand, opp_hand, num_sims=5000)

print("AA vs KK Win Rates:")
print(f"  Win Rate:  {results['Win Rate']:.2%}")
print(f"  Tie Rate:  {results['Tie Rate']:.2%}")
print(f"  Loss Rate: {results['Loss Rate']:.2%}")

Win Rate: 74.41%
Tie Rate: 0.47%
Loss Rate: 25.12%


In [ ]:
## Example 2: Starting Hand Strength Analysis

def analyze_starting_hands(num_hands=50, num_sims=1000):
    """
    Analyze win rates for different starting hand combinations.
    
    Args:
        num_hands: Number of hands to analyze
        num_sims: Simulations per hand
        
    Returns:
        Dictionary mapping hand names to win rates
    """
    strength = {}
    deck = Deck()
    all_hands = list(combinations(deck.cards, 2))
    
    for i, hand in enumerate(all_hands[:num_hands]):
        wins = 0 
        for _ in range(num_sims):
            test_deck = Deck()
            test_deck.cards = [c for c in test_deck.cards if c not in hand]
            test_deck.shuffle()
            opp_hand = test_deck.deal(2)
            community = test_deck.deal(5)
            
            player_rank, _ = best(list(hand) + community)
            opp_rank, _ = best(opp_hand + community)
            
            if player_rank > opp_rank:
                wins += 1
                
        hand_key = f"{hand[0].rank}-{hand[1].rank}"
        strength[hand_key] = wins / num_sims
        
        if (i + 1) % 10 == 0:
            print(f"Progress: {i+1}/{num_hands} hands analyzed")
            
    return strength

# Run analysis
print("Analyzing starting hand strength...")
hand_strength = analyze_starting_hands(num_hands=50, num_sims=500)

# Show top 10 strongest hands
sorted_hands = sorted(hand_strength.items(), key=lambda x: x[1], reverse=True)
print("\nTop 10 Starting Hands by Win Rate:")
for i, (hand, win_rate) in enumerate(sorted_hands[:10], 1):
    print(f"  {i:2d}. {hand:15s} - {win_rate:.2%}")

Progress: 0/1326 hands evaluated.
Progress: 10/1326 hands evaluated.
Progress: 20/1326 hands evaluated.
Progress: 30/1326 hands evaluated.
Progress: 40/1326 hands evaluated.
Progress: 50/1326 hands evaluated.
Progress: 60/1326 hands evaluated.
Progress: 70/1326 hands evaluated.
Progress: 80/1326 hands evaluated.
Progress: 90/1326 hands evaluated.


In [ ]:
## Example 3: Statistical Analysis with Confidence Intervals

def monte_carlo_simulation(player_hand, opp_hand, num_sims=10000, confidence=0.95):
    """
    Run Monte Carlo simulation with confidence intervals.
    
    Args:
        player_hand: Player's hole cards
        opp_hand: Opponent's hole cards
        num_sims: Total simulations
        confidence: Confidence level (default 95%)
        
    Returns:
        Dictionary with statistics
    """
    wins = []
    
    # Run multiple trials
    for trial in range(10):
        trial_wins = 0
        for _ in range(num_sims // 10):
            deck = Deck()
            deck.cards = [c for c in deck.cards if c not in player_hand + opp_hand]
            deck.shuffle()
            community = deck.deal(5)
            
            player_rank, _ = best(player_hand + community)
            opp_rank, _ = best(opp_hand + community)
            
            if player_rank > opp_rank:
                trial_wins += 1
        
        wins.append(trial_wins / (num_sims // 10))
    
    # Calculate statistics
    mean_win = np.mean(wins)
    std_err = stats.sem(wins)
    ci = stats.t.interval(confidence, len(wins)-1, loc=mean_win, scale=std_err)
    
    return {
        'win_rate': mean_win,
        'ci_lower': ci[0],
        'ci_upper': ci[1],
        'std_dev': np.std(wins)
    }

# Run MC simulation
print("Running Monte Carlo simulation...")
player_hand = [Card('Ace', 'Hearts'), Card('King', 'Hearts')]
opp_hand = [Card('Queen', 'Diamonds'), Card('Jack', 'Diamonds')]

mc_results = monte_carlo_simulation(player_hand, opp_hand, num_sims=5000)

print("\nAK vs QJ Simulation Results:")
print(f"  Win Rate: {mc_results['win_rate']:.2%}")
print(f"  95% CI:   [{mc_results['ci_lower']:.2%}, {mc_results['ci_upper']:.2%}]")
print(f"  Std Dev:  {mc_results['std_dev']:.4f}")

Win Rate: 73.85% [95% CI: 73.17% - 74.53%]


In [ ]:
## Example 4: Running a Full Game

Run a complete Texas Hold'em game with multiple players to see how the simulator works end-to-end.

In [ ]:
# Run a complete game
print("=" * 60)
print("TEXAS HOLD'EM GAME")
print("=" * 60)
print()

# Create players
players = [
    Player("Alice", chips=1000),
    Player("Bob", chips=1000),
    Player("Charlie", chips=1000)
]

# Initialize game
game = TexasHoldem(players)

print("Players:")
for p in players:
    print(f"  - {p.name}: {p.chips} chips")
print()

# Deal hole cards
print("=" * 60)
print("DEALING HOLE CARDS")
print("=" * 60)
game.deal_hole_cards()
for p in players:
    print(f"{p.name}: {p.hand}")
print()

# Deal community cards
print("=" * 60)
print("DEALING COMMUNITY CARDS")
print("=" * 60)
game.deal_flop()
print(f"Flop: {game.community_cards}")
game.deal_turn()
print(f"Turn: {game.community_cards}")
game.deal_river()
print(f"River: {game.community_cards}")
print()

# Showdown
print("=" * 60)
print("SHOWDOWN")
print("=" * 60)
winner = game.showdown()